# Load libraries and Themes

In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat)
        library(ggplot2)
        library(tidyverse)
        library(SeuratDisk)
        library(SingleCellExperiment)
        library(DESeq2)
        library(dplyr)
        library(EnhancedVolcano)
        library(Rsamtools)
        library(svglite)
        library(viridis)
        library(harmony)
        library(patchwork)
        library(ggpubr)
        library(pals)
            })})

In [ ]:
#Color Palettes

palette.doublets <- c(
    '#197278',#turqu
    '#C1C1C1'#grey
)

palette.16 <- DiscretePalette(16, palette = "stepped", shuffle = TRUE)

palette.2 <- c(
    "#990902",
    "#0303a3"
)

palette.4 <- DiscretePalette(4, palette = "stepped", shuffle = TRUE)

palette.12 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette.15 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)

palette.dbl <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#000000",#dbl
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)

palette.ex <- c(
    "#bababa",#BC
    "#bababa",#CM_0
    "#bababa",#CM_1
    "#bababa",#CM_2
    "#bababa",#CM_3
    "#bababa",#Cyc
    "#bababa",#EC-cap
    "#000000",#ex
    "#bababa",#FB
    "#bababa",#MESO
    "#bababa",#MP_0
    "#bababa",#MP_1
    "#bababa",#NEU
    "#bababa"#PER
)
    
palette.treatment <- c(
    '#AF0000',#ALDO
    '#C1C1C1'#CTRL
)

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

vln_theme_1 <- theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

vln_theme_2 <- theme(
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LA_CTRL_ALDO")

In [ ]:
set.seed(1234)

# Identify viable clusters and cell types

In [ ]:
obj_harmony_addmodule <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule.h5seurat")

## Cellnumbers per cluster and sample (counts replicates individualy) and subset to equal cell numbers after filtering

In [ ]:
obj_harmony_addmodule@meta.data

In [ ]:
#genes, cells
dim(obj_harmony_addmodule)

summary(colSums(obj_harmony_addmodule))

In [ ]:
number <- table(obj_harmony_addmodule@meta.data$sample_id, 
                          obj_harmony_addmodule@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_sample_id_sgl.csv")

number <- table(obj_harmony_addmodule@meta.data$treatment, 
                          obj_harmony_addmodule@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_Teatment_sgl.csv")

In [ ]:
obj_harmony_addmodule_subset <- subset(obj_harmony_addmodule, seurat_clusters %in% c(6, 13) == FALSE)

In [ ]:
tab <- table(obj_harmony_addmodule_subset@meta.data$sample_id)
print(tab)

In [ ]:
# Downsample to even cell numbers
downsampled_list <- list()

# Get unique sample IDs
sample_ids <- unique(obj_harmony_addmodule_subset@meta.data$sample_id)

# Loop through each sample ID
for (sample_id in sample_ids) {
  # Subset the object for the current sample_id
  subset_obj <- subset(obj_harmony_addmodule_subset, subset = sample_id == !!sample_id)

  # Downsample to a maximum of 7789 cells
  if (ncol(subset_obj) > 7789) {
    # Randomly select 7789 cells
    cell_indices <- sample(colnames(subset_obj), 7789)
    subset_obj <- subset(subset_obj, cells = cell_indices)
  }
  
  # Store the downsampled object in the list
  downsampled_list[[sample_id]] <- subset_obj
}

# Combine all downsampled objects back into one Seurat object
obj_harmony_addmodule_subset_down <- merge(downsampled_list[[1]], y = downsampled_list[-1])

# Check the counts of cells per sample_id
tab <- table(obj_harmony_addmodule_subset_down@meta.data$sample_id)
print(tab) 

In [ ]:
number <- table(obj_harmony_addmodule_subset_down@meta.data$sample_id, 
                          obj_harmony_addmodule_subset_down@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_sample_id_sgl_evencellnumbers.csv")

number <- table(obj_harmony_addmodule_subset_down@meta.data$treatment, 
                          obj_harmony_addmodule_subset_down@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_Teatment_sgl_evencellnumbers.csv")

In [ ]:
obj <- NormalizeData(obj_harmony_addmodule_subset_down, verbose = FALSE)
obj <- FindVariableFeatures(obj, verbose = FALSE)
obj <- ScaleData(obj, verbose = FALSE)
obj <- RunPCA(obj, assay = "RNA", npcs = 35, verbose = FALSE)

In [ ]:
# integrate "sex"
obj$sex <- as.factor(obj$sex)

obj <- obj %>%
  RunHarmony(group.by.vars = c("sex"), plot_convergence = TRUE, assay.use = "RNA", verbose = FALSE)

In [ ]:
#Run UMAP
obj <- obj %>%
  RunUMAP(reduction = "harmony", dims = 1:35, verbose = FALSE) %>%
  FindNeighbors(reduction = "harmony", dims = 1:35, verbose = FALSE) %>%
  FindClusters(resolution = 0.25)

SaveH5Seurat(obj, "seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj, reduction = "umap", label = TRUE, label.size = 5, shuffle = TRUE, group.by = "seurat_clusters", cols = palette.16) + umap_theme + NoLegend() + theme(text = element_text(size = 10))
DimPlot(obj, reduction = "umap", label = FALSE, label.size = 5, shuffle = TRUE, group.by = "treatment", cols = palette.treatment) + umap_theme + theme(text = element_text(size = 10))

## FindAllMarkers

In [ ]:
all.markers <- FindAllMarkers(obj, only.pos = TRUE, min.pct = 0.50, logfc.threshold = 0.58, test.use = "wilcox")
all.markers %>%
  group_by("seurat_clusters") %>%
  slice_max(n = 50, order_by = avg_log2FC)

write.csv(all.markers, file = "DEGs/all.markers_wilcox_downsaple.csv")

## Plots of markers

In [ ]:
stress_markers <- list(
    "stress" = c("Ctgf", "Nppb", "Nppa")
    )

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 300)

DOT_stress_markers <- DotPlot(
  obj,
  features = unlist(stress_markers, use.names = FALSE)) +
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")), 
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev)

ggsave("Plots/DOT_stress_markers.svg", width = 9, height = 10, units = "cm", dpi = 300)

DOT_stress_markers

In [ ]:
myeloid_markers <- list(
  "Monocytes" = c("Ccr2", "Cd14", "Fcgr3"),
  "Macrophages" = c("Cd68", "Itgam", "Adgre1"),
  "Dendritic Cells" = c("Flt3", "Clec10a", "Fcer1a"),
  "Neutrophils" = c("Cxcr1", "Cxcr2", "Lcn2"),
  "Eosinophils" = c("Epx", "Prg2", "Il5ra"),
  "Basophils" = c("Il4", "Mcpt8", "Ms4a2"), 
  "Mast Cells" = c("Kit", "Mcpt1", "Mcpt4"),
  "Microglia" = c("Cx3cr1", "Tmem119", "P2ry12"),
  "Cycling" = c("Mki67", "Top2a", "Pcna")
)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 300)

DOT_myeloid_markers <- DotPlot(
  obj,
  features = unlist(myeloid_markers, use.names = FALSE)) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev)

num_features <- length(unlist(myeloid_markers, use.names = FALSE)) 
vline_positions <- seq(3.5, num_features, by = 3)

DOT_myeloid_markers <- DOT_myeloid_markers + 
  geom_vline(xintercept = vline_positions, color = "black", linetype = "solid", size = 0.5)

ggsave("Plots/DOT_myeloid_markers.svg", width = 20, height = 11, units = "cm", dpi = 300)

DOT_myeloid_markers

In [ ]:
canonical_markers <- list(
    "Endothelial Cell" = c("Cdh5", "Pecam1", "Npr3", "Kdr", "Vwf"),
    "Cardiomyocyte" = c("Myh6", "Pln", "Atp2a2", "Ryr2", "Kcnj3"),
    "Fibroblast" = c("Col1a1", "Col3a1", "Dcn", "Pdgfra", "Lamc1"),
    "Macrophage" = c("Lyz2", "Maf", "F13a1", "Arhgap15", "Runx1"),
    "T-Cell" = c("Lat", "Skap1", "Cd247", "Il7r", "Grap2"),
    "B-Cell" = c("Cd79b", "H2-Ob", "Bank1", "Cd55", "Pax5"),
    "Smooth Muscle Cell" = c("Acta2", "Myh11", "Lmod1", "Sema5a", "Kcnma1"),
    "DC-like Cell" = c("Plbd1", "Ccr2", "Naaa", "Ear2", "H2afy"),
    "Granulocyte" = c("S100a8", "Csf3r", "Lmnb1", "Hp", "Hdc"),
    "NK Cell" = c("Ccl5", "Nkg7", "Ctsw", "Gzma", "Cd27"),
    "Pericyte" = c("Kcnj8", "Vtn", "Colec11", "Pdgfrb", "Abcc9"),
    "Schwann Cell" = c("Nrxn1", "Nrn1", "Plp1", "Cnp", "Cd59a"))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 300)

DOT_canonical_markers <- DotPlot(
  obj,
  features = unlist(canonical_markers, use.names = FALSE)) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev)

num_features <- length(unlist(canonical_markers, use.names = FALSE)) 
vline_positions <- seq(5.5, num_features, by = 5)

DOT_canonical_markers <- DOT_canonical_markers + 
  geom_vline(xintercept = vline_positions, color = "black", linetype = "solid", size = 0.5)

ggsave("Plots/DOT_canonical_markers.svg", width = 38, height = 11, units = "cm", dpi = 300)

DOT_canonical_markers

In [ ]:
Top5_markers <- list(
    "0" = c("Kcnj3", "Erbb4", "Ryr2", "Rbm20", "Chrm2"),
    "1" = c("Npr3", "Tmem2", "Chrm3", "Cgnl1", "Wdr17"),
    "2" = c("Ebf2", "Pcdh9", "Abca8a", "Lsamp", "Gsn"),
    "3" = c("F13a1", "Arhgap15", "Dock2", "Mrc1", "Runx1"),
    "4" = c("Myo10", "Fabp4", "Ccdc85a", "Ablim3", "Meox2"),
    "5" = c("Notch3", "Trpc3", "Rgs5", "Ano1", "Abcc9"),
    "6" = c("Muc16", "Pcdh15", "Wt1", "Dpp4", "Sema3d"),
    "7" = c("Slc9a9", "Prkcb", "Ptprc", "Mctp1", "Dock8"),
    "8" = c("Acer2", "Htra3", "C1qtnf7", "Tnxb", "Col15a1"),
    "9" = c("Cryab", "Myl7", "Mb", "Csrp3", "Fabp3"),
    "10" = c("Dock10", "Iqgap2", "Diaph3", "Ikzf1", "Ankrd44"),
    "11" = c("Mmrn1", "Reln", "Flt4", "Pard6g", "Prox1"),
    "12" = c("Csmd1", "Slc35f1", "Cadm2", "Cdh19", "Chl1"),
    "13" = c("Nppa", "Myh6", "Ttn", "Tnnt2", "Fgf12"),
    "14" = c("Pax5", "Bank1", "Ikzf3", "Ighm", "Btla"))

In [ ]:
# Check for duplicates in "Top5_markers" 
marker_df <- data.frame(
  Group = rep(names(Top5_markers), times = sapply(Top5_markers, length)),
  Marker = unlist(Top5_markers)
)

duplicated_markers <- marker_df$Marker[duplicated(marker_df$Marker)]

duplicated_info <- marker_df[marker_df$Marker %in% duplicated_markers, ]

print(duplicated_info)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 300)

DOT_Top5_markers <- DotPlot(
  obj,
  features = unlist(Top5_markers, use.names = FALSE)) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  #scale_colour_viridis_c(option = "D", direction = 1) +  
  scale_y_discrete(limits = rev)

num_features <- length(unlist(Top5_markers, use.names = FALSE)) 
vline_positions <- seq(5.5, num_features, by = 5)

DOT_Top5_markers <- DOT_Top5_markers + 
  geom_vline(xintercept = vline_positions, color = "black", linetype = "solid", size = 0.5)

ggsave("Plots/DOT_Top5_markers.svg", width = 45, height = 11, units = "cm", dpi = 300)

DOT_Top5_markers

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 30, repr.plot.res = 300)

# Generate your feature plots as a list
UMAP_grid_canonical_marker <- FeaturePlot(
  obj,
  features = unlist(Top5_markers),
  combine = FALSE,  
  cols = viridis(256),
  max.cutoff = "q98", ncol = 5
)

# Customize each plot with your theme and settings
for (i in 1:75) {
  UMAP_grid_canonical_marker[[i]] <- UMAP_grid_canonical_marker[[i]] + 
    umap_theme + 
    NoLegend() + 
    theme(text = element_text(size = 15))
}

# Combine all plots into a grid using patchwork's wrap_plots()
combined_plot <- wrap_plots(UMAP_grid_canonical_marker, ncol = 5)

# Save the combined plot
ggsave("Plots/UMAP_grid_canonical_marker.svg", plot = combined_plot, width = 25, height = 75, units = "cm", dpi = 200)

# Display the combined plot
combined_plot

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj, reduction = "umap", label = TRUE, label.size = 5, group.by = "seurat_clusters", cols = palette.15) + umap_theme + NoLegend() + theme(text = element_text(size = 10))

In [ ]:
# Top10 markers for EC subtypes from Carmeliet Lab
Top10EC_markers <- list(
"artery" <- c("Fbln5", "Stmn2", "8430408G22Rik", "Glul", "Fos", "Id1", "Hey1", "Sox17", "Gadd45g", "Btg2"),
"capillary arterial" <- c("Cxcl12", "Rbp7", "Mgll", "Ly6c1", "Aqp7", "Btnl9", "Pdgfd", "Sema7a", "Clec2d", "Unc5b"),
"capillary" <- c("Ssh2", "Endou", "Fam57b", "Kifc3", "Ntf3", "Tmem182", "Fam212b", "4430402I18Rik", "Coro2b", "Fam117b"),
"capillary venous" <- c("Vcam1", "Ier3", "Pltp", "Pi16", "Fmo2", "Fmo1", "Calcrl", "Emp1", "AU021092", "Bsg"),
"large vein" <- c("Emp2", "Mgp", "Cfh", "Apoe", "Cpe", "Cytl1", "Bgn", "Plvap", "Dcn", "Ctsh"),
"interferon" <- c("Rgcc", "Lpl", "Isg15", "Car4", "Ifit3", "Rtp4", "Ifi203", "Rsad2", "Tcf15", "Ifit1"),
"angiogenic" <- c("Tmsb10", "Ift122", "Sparc", "Cldn5", "Col4a2", "Apln", "Vim", "Igfbp7", "Vwa1", "Mest"),
"lymphatic" <- c("Ccl21a", "Mmrn1", "Fgl2", "Prss23", "Thy1", "Igfbp5", "Fth1", "Lyve1", "Prelp", "Prox1"))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5, repr.plot.res = 300)

DOT_Top10EC_markers <- DotPlot(
  obj,
  features = unlist(Top10EC_markers, use.names = FALSE)
) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev)

num_features <- length(unlist(Top10EC_markers, use.names = FALSE)) 
vline_positions <- seq(10.5, num_features, by = 10)

DOT_Top10EC_markers <- DOT_Top10EC_markers + 
  geom_vline(xintercept = vline_positions, color = "black", linetype = "solid", size = 0.5)

ggsave("Plots/DOT_Top10EC_markers_Carmeliet.svg", width = 48, height = 13, units = "cm", dpi = 300)

# Print the plot
print(DOT_Top10EC_markers)

In [ ]:
MΦ_markers <- list(
  "M1 (pro-inflammatory)" = c("Cd80", "Cd86", "Nos2", "Tnf", "Il6", "Il12b", "Cxcl9", "Ccl2"),
  "M2 (anti-inflammatory)" = c("Cd163", "Mrc1", "Arg1", "Il10", "Tgfb1", "Mmp9", "Vegfa", "Retnla"))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5, repr.plot.res = 300)

DOT_MΦ_markers <- DotPlot(
  obj,
  features = unlist(MΦ_markers, use.names = FALSE)
) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev)

num_features <- length(unlist(MΦ_markers, use.names = FALSE)) 
vline_positions <- seq(8.5, num_features, by = 8)

DOT_MΦ_markers <- DOT_MΦ_markers + 
  geom_vline(xintercept = vline_positions, color = "black", linetype = "solid", size = 0.5)

#ggsave("Plots/DOT_MΦ_markers.svg", width = 48, height = 13, units = "cm", dpi = 300)

# Print the plot
print(DOT_MΦ_markers)

## Rename Cluster annotations

In [ ]:
obj <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample.h5seurat")

In [ ]:
cluster_annotations <- list(
    "0" = "CM_0",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "7" = "dbl",
    "8" = "dbl",
    "9" = "CM_1",
    "10" = "TC",
    "11" = "EC-lym",
    "12" = "SC",
    "13" = "CM_2",
    "14" = "BC"
)

Cannot merge two objects with different Idents properly. Celltype is stored in seperate meta.data column

In [ ]:
cluster_annotations <- unlist(cluster_annotations, use.names = FALSE)

obj@meta.data$Ident_numerical <- Idents(obj)
names(cluster_annotations) <- levels(obj)
obj <- RenameIdents(obj, cluster_annotations)
obj@meta.data$cell_type <- Idents(obj)
Idents(obj) <- obj@meta.data$"Ident_numerical"

SaveH5Seurat(obj, "seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample_rename_dbl")

In [ ]:
obj
sort(unique(Idents(obj)))
sort(unique(obj@meta.data$cell_type))

In [ ]:
#cellnumbers after rename
number <- table(obj@meta.data$sample_id, 
                          obj@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_sample_id_sgl_evencellnumbers_rename.csv")

number <- table(obj@meta.data$treatment, 
                          obj@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_Teatment_sgl_evencellnumbers_rename.csv")

## Visualize QC and UMAP

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 300)

VlnPlot_QC <- VlnPlot(obj, group.by = "cell_type", 
                      features = c("nCount_RNA", "nFeature_RNA", "percent.mt", "scDblFinder.weighted"),
                      pt.size = 0, raster = TRUE, ncol = 2, cols = palette.dbl)

plots <- lapply(1:4, function(i) {
  if (i == 4) {
    VlnPlot_QC[[i]] + theme(
      axis.text.x = element_text(size = 15),  
      axis.text.y = element_text(size = 15),  
      axis.ticks.x = element_line(),   
      axis.title.x = element_blank(),
      plot.title = element_text(size = 15)  
    )
  } else {
    VlnPlot_QC[[i]] + theme(
      axis.text.x = element_blank(), 
      axis.text.y = element_text(size = 15),  
      axis.ticks.x = element_blank(), 
      axis.title.x = element_blank(),
      plot.title = element_text(size = 15)  
    )
  }
})

combined_plot <- wrap_plots(plots, ncol = 2)

ggsave("Plots/VlnPlot_QC_withdblcluster.svg", combined_plot, units = "cm", dpi = 300, width = 30, height = 20)

suppressWarnings(print(combined_plot))

## Remove left-over doublet clusters

In [ ]:
obj_subset <- subset(obj, subset = cell_type %in% c("dbl"), invert = TRUE)

In [ ]:
SaveH5Seurat(obj_subset, "seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample_rename")

In [ ]:
#Final cellnumbers
number <- table(obj_subset@meta.data$sample_id, 
                          obj_subset@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_sample_id_final.csv")

number <- table(obj_subset@meta.data$treatment, 
                          obj_subset@meta.data$seurat_clusters)
write.csv(number, file = "cellnumbers/number_perCluster_Teatment_final.csv")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 300)

VlnPlot_QC <- VlnPlot(obj_subset, group.by = "cell_type", 
                      features = c("nCount_RNA", "nFeature_RNA", "percent.mt", "scDblFinder.weighted"),
                      pt.size = 0, raster = TRUE, ncol = 2, cols = palette)

plots <- lapply(1:4, function(i) {
  if (i == 4) {
    VlnPlot_QC[[i]] + theme(
      axis.text.x = element_text(size = 15),  
      axis.text.y = element_text(size = 15),  
      axis.ticks.x = element_line(),   
      axis.title.x = element_blank(),
      plot.title = element_text(size = 15)  
    )
  } else {
    VlnPlot_QC[[i]] + theme(
      axis.text.x = element_blank(), 
      axis.text.y = element_text(size = 15),  
      axis.ticks.x = element_blank(), 
      axis.title.x = element_blank(),
      plot.title = element_text(size = 15)  
    )
  }
})

combined_plot <- wrap_plots(plots, ncol = 2)

ggsave("Plots/VlnPlot_QC.svg", combined_plot, units = "cm", dpi = 300, width = 30, height = 20)

suppressWarnings(print(combined_plot))

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 2, repr.plot.res = 300)

UMAP_rename_seurat_clusters <- DimPlot(obj_subset, group.by = "seurat_clusters", label = TRUE, shuffle = TRUE, cols = palette.23) +
  umap_theme + ggtitle("UMAP seurat clusters") + theme(text = element_text(size = 15)) & NoLegend()

UMAP_rename_cell_type <- DimPlot(obj_subset, group.by = "cell_type", label = TRUE, shuffle = TRUE, cols = palette) +
  umap_theme + ggtitle("UMAP cell types") + theme(text = element_text(size = 15)) & NoLegend()

UMAP_rename_treatment <- DimPlot(obj_subset, label = FALSE, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) +
  umap_theme + ggtitle("UMAP treatment") + theme(text = element_text(size = 15)) & NoLegend()

UMAP_rename_sample_id <- DimPlot(obj_subset, label = FALSE, group.by = "sample_id", shuffle = TRUE, cols = palette.4) +
  umap_theme + ggtitle("UMAP sample_id") + theme(text = element_text(size = 15)) & NoLegend()

ggsave("Plots/UMAP_rename_seurat_clusters.svg", UMAP_rename_seurat_clusters, units = "cm", dpi = 300, width = 30, height = 20)
ggsave("Plots/UMAP_rename_cell_type.svg", UMAP_rename_cell_type, units = "cm", dpi = 300, width = 30, height = 20)
ggsave("Plots/UMAP_rename_treatment.svg", UMAP_rename_treatment, units = "cm", dpi = 300, width = 30, height = 20)
ggsave("Plots/UMAP_rename_sample_id.svg", UMAP_rename_sample_id, units = "cm", dpi = 300, width = 30, height = 20)

UMAP_rename_seurat_clusters
suppressWarnings(print(UMAP_rename_cell_type))
UMAP_rename_treatment
UMAP_rename_sample_id

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 2, repr.plot.res = 300)

UMAP_dbl_clusters <- DimPlot(obj, group.by = "cell_type", label = FALSE, cols = palette.ex) +
  umap_theme + ggtitle("UMAP excluded dbl clusters") + theme(text = element_text(size = 15)) + NoLegend()

ggsave("Plots/UMAP_excluded_dbl_clusters.svg", UMAP_dbl_clusters, units = "cm", dpi = 300, width = 30, height = 20)

suppressWarnings(print(UMAP_dbl_clusters))

In [ ]:
Top5_markers <- list(

"0" = c("Kcnj3", "Erbb4", "Ryr2", "Rbm20", "Chrm2"),
"1" = c("Npr3", "Tmem2", "Chrm3", "Cgnl1", "Wdr17"),
"2" = c("Ebf2", "Pcdh9", "Abca8a", "Lsamp", "Gsn"),
"3" = c("F13a1", "Arhgap15", "Dock2", "Mrc1", "Runx1"),
"4" = c("Myo10", "Fabp4", "Ccdc85a", "Ablim3", "Meox2"),
"5" = c("Notch3", "Trpc3", "Rgs5", "Ano1", "Abcc9"),
"6" = c("Muc16", "Pcdh15", "Wt1", "Dpp4", "Sema3d"),
"9" = c("Cryab", "Myl7", "Mb", "Csrp3", "Fabp3"),
"10" = c("Dock10", "Iqgap2", "Diaph3", "Ikzf1", "Ankrd44"),
"11" = c("Mmrn1", "Reln", "Flt4", "Pard6g", "Tc2n"),
"12" = c("Csmd1", "Slc35f1", "Cadm2", "Cdh19", "Chl1"),
"13" = c("Nppa", "Myh6", "Ttn", "Tnnt2", "Fgf12"),
"14" = c("Pax5", "Bank1", "Ikzf3", "Ighm", "Btla"))


In [ ]:
cluster_annotations_sgl <- list(
    "0" = "CM_0",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "9" = "CM_1",
    "10" = "TC",
    "11" = "EC-lym",
    "12" = "SC",
    "13" = "CM_2",
    "14" = "BC"
)

In [ ]:
names(cluster_annotations_sgl) <- levels(obj_subset)
obj_subset <- RenameIdents(obj_subset, cluster_annotations_sgl)

DOT_Top5_markers_sgl <- DotPlot(
  obj_subset,
  features = unlist(Top5_markers, use.names = FALSE)) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  #scale_colour_viridis_c(option = "D", direction = 1) +  
  scale_y_discrete(limits = rev)

num_features <- length(unlist(Top5_markers, use.names = FALSE)) 
vline_positions <- seq(5.5, num_features, by = 5)

DOT_Top5_markers_sgl <- DOT_Top5_markers_sgl + 
  geom_vline(xintercept = vline_positions, color = "black", linetype = "solid", size = 0.5)

Idents(obj_subset) <- obj_subset@meta.data$"Ident_numerical"

ggsave("Plots/DOT_Top5_markers_sgl.svg", width = 40, height = 11, units = "cm", dpi = 300)

suppressWarnings(print(DOT_Top5_markers_sgl))

# Features with non-zero counts 

In [ ]:
cluster_annotations_sgl <- list(
    "0" = "CM_0",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "9" = "CM_1",
    "10" = "TC",
    "11" = "EC-lym",
    "12" = "SC",
    "13" = "CM_2",
    "14" = "BC"
)

In [ ]:
export_non_zero_counts <- function(seurat_object, LA_LV, cluster_annotations) { 
    # Filter the Seurat object for the specified chamber and cell type 
    seurat_table <- subset(x = seurat_object, subset = chamber == LA_LV) 
    # Iterate over cluster annotations and export non-zero counts 
    for (cluster_id in names(cluster_annotations)) { 
        cell <- cluster_annotations[[cluster_id]] 
        seurat_table_filtered <- subset(x = seurat_table, subset = cell_type == cell) 
        # Get the counts for the filtered Seurat object 
        counts <- GetAssayData(seurat_table_filtered, assay = "RNA", layer = "counts") 
        # Find non-zero counts 
        nonzero <- as.data.frame(rowSums(counts) > 0) 
        names(nonzero)[names(nonzero) == "rowSums(counts) > 0"] <- "nonzerofeature"
        nonzero <- filter(nonzero, nonzerofeature == TRUE)
        nonzero$names <- rownames(nonzero)
        nonzero$nonzerofeature <- NULL

        # Export to CSV 
        filename <- paste0("nonzerocounts/nonzerocounts_", LA_LV, "_", cell, ".csv") 
        write.csv(nonzero, file = filename, row.names = FALSE) 
    } 
}

In [ ]:
export_non_zero_counts(obj_subset, "LA", cluster_annotations_sgl)

# Combine CM clusters and save final object

In [ ]:
# Combine CM clusters 
cluster_annotations_CMcomb <- list(
    "0" = "CM",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "7" = "CM",
    "8" = "TC",
    "9" = "EC-lym",
    "10" = "SC",
    "11" = "CM",
    "12" = "BC")

In [ ]:
names(cluster_annotations_CMcomb) <- levels(obj_subset)
obj_subset <- RenameIdents(obj_subset, cluster_annotations_CMcomb)

obj_subset$"cell_type_CMcomb" <- Identsobj_subsetobj.full)

In [ ]:
saveRDS(obj_subset, file = "seurat_objects/LA_snRNAseq_final.rds")

In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.1 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/scrna_dm/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] pals_1.9                    ggpubr_0.6.0               
 [3] patchwork_1.3.0             harmony_1.2.3              
 [5] Rcpp_1.0.13-1               viridis_0.6.5              
 [7] viridisLite_0.4.2           svglite_2.1.3              
 [9] Rsamtools_2.18.0